In [6]:
import torch

from data_reader import *

[(x['Date'], x['Adjusted']) for x in nasdaq[:10]], [(x['Date'], x['Adjusted']) for x in stockAAPL[:10]]

([('2015-01-02', 4726.81005859375),
  ('2015-01-05', 4652.56982421875),
  ('2015-01-06', 4592.740234375),
  ('2015-01-07', 4650.47021484375),
  ('2015-01-08', 4736.18994140625),
  ('2015-01-09', 4704.06982421875),
  ('2015-01-12', 4664.7099609375),
  ('2015-01-13', 4661.5),
  ('2015-01-14', 4639.31982421875),
  ('2015-01-15', 4570.81982421875)],
 [('2015-01-05', 23.63528251647949),
  ('2015-01-06', 23.63751220703125),
  ('2015-01-07', 23.968963623046875),
  ('2015-01-08', 24.8899040222168),
  ('2015-01-09', 24.91659545898437),
  ('2015-01-12', 24.302640914916992),
  ('2015-01-13', 24.518415451049805),
  ('2015-01-14', 24.42498588562012),
  ('2015-01-15', 23.762081146240234),
  ('2015-01-16', 23.57744789123535)])

## 纳斯达克指数数据解释
Date: 2015-01-02，表示交易日期。
Ticker: IXIC，代表纳斯达克综合指数。
Open: 4760.240234375，开盘价。
High: 4777.009765625，当天最高价。
Low: 4698.10986328125，当天最低价。
Close: 4726.81005859375，收盘价。
Adjusted: 4726.81005859375，调整后的收盘价（考虑分红、拆股等因素）。
Returns: nan，收益率（此处为缺失值）。
Volume: 1435150000，交易量。
## 苹果公司股票数据解释
Date: 2015-01-05，交易日期。
Ticker: AAPL，代表苹果公司。
Open: 27.07250022888184，开盘价。
High: 27.162500381469727，当天最高价。
Low: 26.352500915527344，当天最低价。
Close: 26.5625，收盘价。
Adjusted: 23.63528251647949，调整后的收盘价。
Returns: -0.0281718672358495，收益率（表示相对于前一日的涨跌比率）。
Volume: 257142000，交易量。

# 输入特征

5-day 自回归

# 特征建模

[开盘价，最高价-开盘价，最低价-开盘价，收盘价-开盘价，交易量]

In [27]:
from torch import nn
from deeplotx import LinearRegression

class Autoregression(nn.Module):
    def __init__(self, features: int, hidden_size: int, model_name: str | None = None):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=features, 
            hidden_size=hidden_size, 
            num_layers=3, 
            batch_first=True,
            bidirectional=True
        )
        self.ffn = LinearRegression(
            input_dim=hidden_size * 2, 
            output_dim=features, 
            model_name=model_name
        )
    
    def initial_state(self, num_directions: int = 2, batch_size: int = 1) -> tuple[torch.Tensor, ...]:
        zeros = torch.zeros(self.lstm.num_layers * num_directions, batch_size, self.lstm.hidden_size)
        return tuple(2 * [zeros])
    
    def forward(self, x: torch.Tensor, state: tuple):
        x, (hidden_state, cell_state) = self.lstm(x, state)
        x = self.ffn(x[-1, -1, :])
        return x, hidden_state, cell_state

model = Autoregression(features=10, hidden_size=15)
model.forward(torch.randn(1, 1, 10), model.initial_state())[0].shape

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]), tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]))
torch.Size([1, 1, 30])


torch.Size([10])